In [1]:
from pandas_datareader import data as pdr 
from datetime import date
import yfinance as yf 
yf.pdr_override()
import pandas as pd
import matplotlib.pyplot as plt 
import math
import quandl
import numpy as np
from sklearn import preprocessing
%matplotlib qt

c:\users\juand\appdata\local\programs\python\python36\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [14]:
ticker_sp = '^GSPC'
ticker_gold = 'GC=F'
ticker_oil = 'CL=F'
ticker_dax = '^GDAXI'
ticker_nikkei = '^N225'
ticker_ftse = '^FTSE'
ticker_shanghai = '000001.SS'

auth_tok = "Nv1rJgRR7u88iz_dg7Y6"

end_date = "2020-07-1"
start_date = "1970-01-02"

In [15]:
def getGOLDData ():
    # Contains only price from 1975 onwards
    data = quandl.get("CHRIS/CME_GC1", trim_start = start_date, trim_end = end_date, authtoken=auth_tok)
    data = data[['Last']]
    data.columns = ["GOLD Adj Close"]
    return data

def getSPData():
    # Contains price from 1970 onwards
    data = pdr.get_data_yahoo(ticker_sp, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["SP500 Adj Close"]
    return data

def getDAXData():
    # Contains price from 1988 onwards
    data = pdr.get_data_yahoo(ticker_dax, start=start_date, end=end_date)
    data = data[data.columns[4:5]]
    data.columns = ["DAX Adj Close"]
    return data


def getOILData():
    # Contains only price FROM 1984 onwards
    data = quandl.get("CHRIS/CME_CL1", trim_start = start_date, trim_end = end_date, authtoken=auth_tok)
    data = data[["Last"]]
    data.columns=["OIL Adj Close"]
    return data


def getNIKKEIData():
    # Contains only price from 1970 onwards
    data = pdr.get_data_yahoo(ticker_nikkei, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["NIKKEI Adj Close"]
    return data


def getFTSEData():
    # Contains price from 1984 onwards
    data = pdr.get_data_yahoo(ticker_ftse, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["FTSE Adj Close"]
    return data

def getSHANGHAIData():
    # Contains only price from 1997
    data = pdr.get_data_yahoo(ticker_shanghai, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["SHANGHAI Adj Close"]
    return data

In [25]:
plt.plot(getSHANGHAIData())
plt.show()

[*********************100%***********************]  1 of 1 completed


In [26]:
def combineData():
    allData = [getSPData(), getDAXData(), getFTSEData(), getGOLDData(), getOILData()]
    mergedData = pd.concat(allData, axis = 1)
    cleanData = mergedData.dropna()
    return cleanData

In [60]:
data = combineData()
print(data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
            SP500 Adj Close  DAX Adj Close  FTSE Adj Close  GOLD Adj Close  \
Date                                                                         
1987-12-30       247.860001    1005.190002     1759.800049           485.5   
1988-01-04       255.940002     956.489990     1713.900024           480.5   
1988-01-05       258.630005     996.099976     1789.599976           483.2   
1988-01-06       258.890015    1006.010010     1787.099976           485.3   
1988-01-07       261.070007    1014.469971     1787.199951           483.1   
...                     ...            ...             ...             ...   
2020-06-03      3122.870117   12487.360352     6382.399902          1694.0   
2020-06-04      3112.350098   12430.559570     6341.399902          1712.3   
2020-06-05   

In [61]:
FUTURE_TO_PREDICT = 1

data['Future'] = data['SP500 Adj Close'].shift(-FUTURE_TO_PREDICT)

data.dropna(inplace=True)

print(data)

            SP500 Adj Close  DAX Adj Close  FTSE Adj Close  GOLD Adj Close  \
Date                                                                         
1987-12-30       247.860001    1005.190002     1759.800049           485.5   
1988-01-04       255.940002     956.489990     1713.900024           480.5   
1988-01-05       258.630005     996.099976     1789.599976           483.2   
1988-01-06       258.890015    1006.010010     1787.099976           485.3   
1988-01-07       261.070007    1014.469971     1787.199951           483.1   
...                     ...            ...             ...             ...   
2020-06-02      3080.820068   12021.280273     6220.100098          1724.4   
2020-06-03      3122.870117   12487.360352     6382.399902          1694.0   
2020-06-04      3112.350098   12430.559570     6341.399902          1712.3   
2020-06-05      3193.929932   12847.679688     6484.299805          1679.5   
2020-06-08      3232.389893   12819.589844     6472.600098      

In [62]:
def buy_or_sell (current, future):
    if (future > current):
        return 1
    else:
        return 0
    
data['Target'] = list(map(buy_or_sell, data['SP500 Adj Close'], data['Future']))

In [63]:
NUMBER_OF_DATA_POINTS = len(data)
SIZE_TRAINING = int(NUMBER_OF_DATA_POINTS * 0.85)
SIZE_TESTING  = NUMBER_OF_DATA_POINTS - SIZE_TRAINING
print("size of training data: {}".format(SIZE_TRAINING))
print("size of testing data: {}".format(SIZE_TESTING))

data_training = data[:SIZE_TRAINING]
data_testing  = data[SIZE_TRAINING:]

size of training data: 6675
size of testing data: 1179


In [64]:
print(data_training)

            SP500 Adj Close  DAX Adj Close  FTSE Adj Close  GOLD Adj Close  \
Date                                                                         
1987-12-30       247.860001    1005.190002     1759.800049           485.5   
1988-01-04       255.940002     956.489990     1713.900024           480.5   
1988-01-05       258.630005     996.099976     1789.599976           483.2   
1988-01-06       258.890015    1006.010010     1787.099976           485.3   
1988-01-07       261.070007    1014.469971     1787.199951           483.1   
...                     ...            ...             ...             ...   
2015-06-17      2100.439941   10978.009766     6680.600098          1176.4   
2015-06-18      2121.239990   11100.299805     6707.899902          1201.5   
2015-06-19      2109.989990   11040.099609     6710.500000          1201.5   
2015-06-22      2122.850098   11460.500000     6825.700195          1183.7   
2015-06-23      2124.199951   11542.540039     6834.899902      

In [65]:
def process_data(data):
    data.drop("Future", axis = 1, inplace = True)  # Drop the future column so that the NN doesn't have access to the future
    
    for column in data.columns:  # Normalize the columns
        if column != "Target":   # We only want to normalize the other columns
            data[column] = data[column].pct_change()   # Normalization by percent change
            print(data[column].values[:20])
            data.dropna(inplace = True)
            data[column] = preprocessing.scale(data[column].values)    # Scale values between 0 and 1
            print(data[column].values[:20])
    data.dropna(inplace = True)
    
    return data

In [66]:
data_training = process_data(data_training)
data_testing  = process_data(data_testing)

[        nan  0.03259906  0.01051029  0.00100533  0.00842054 -0.06768305
  0.01680366 -0.008364    0.00158911  0.0002848   0.02509353 -0.00067446
 -0.01016356 -0.02683299  0.00210194  0.0138192   0.02300202 -0.01031047
 -0.00076132  0.01567884]
[ 2.82820692  0.88882876  0.05430093  0.70535027 -5.97648971  1.44138286
 -0.76832028  0.10555597 -0.00896129  2.16922751 -0.09318404 -0.92632004
 -2.38988445  0.15058236  1.17934922  1.9855942  -0.9392185  -0.10081003
  1.34262388  1.27632   ]
[        nan  0.04141181  0.00994883  0.00840942  0.01204567 -0.03815166
 -0.00063797 -0.02140056  0.008998   -0.02257661  0.05319907 -0.0228784
 -0.02013915 -0.01166133  0.01816189 -0.0039835   0.00584857 -0.00887161
 -0.02283074 -0.00187682]
[ 2.80962854  0.64998229  0.54431553  0.79391089 -2.65167728 -0.07670498
 -1.50186718  0.58471643 -1.58259227  3.61871574 -1.60330786 -1.41528305
 -0.83335854  1.21373339 -0.30634548  0.36853625 -0.64186954 -1.60003606
 -0.16174117  1.10376668]
[            nan -1.3

c:\users\juand\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
c:\users\juand\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\juand\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [72]:
print(preprocessing.scale([-2,5,10]))

[-1.28684238  0.13545709  1.15138528]


In [71]:
print(data_testing['DAX Volume'].pct_change().values)

[            nan -2.64913801e-01 -1.18269126e-01 -3.18518858e-02
 -2.10300316e-01  4.37250208e-01 -9.08230796e-02 -8.19031289e-03
  1.83622461e-01 -4.40955768e-02 -5.17850901e-03  2.84846794e-01
  7.48576455e-01 -5.08281710e-01 -4.17764244e-02  1.54129305e-01
  6.25127249e-03 -1.62541903e-01 -6.97182372e-02  2.91522312e-01
  1.98971125e-02 -6.47107637e-02  1.24215994e-01 -4.57583271e-02
 -4.47311713e-02  6.04524598e-02 -2.04612634e-01 -7.75977193e-02
  3.56396961e-01  1.33408908e-01 -3.89966622e-01  2.08468195e-01
 -1.90093679e-01 -1.93221908e-01  2.53219821e-01  8.81422925e-01
 -3.56214459e-01  9.48149772e-03 -7.65941082e-02  5.09881834e-01
 -2.08986560e-01 -7.16012360e-02  3.87942177e-02 -2.07525660e-01
  3.94679069e-01  1.53240658e-01  8.09561711e-01 -6.40673642e-01
  1.39025576e-02 -1.15796494e-01  2.74471974e-02  6.03484783e-02
 -1.02945929e-01  5.67171676e-01 -3.19585760e-02  1.14568421e-01
 -1.00657638e-01 -2.57832235e-01  7.21461531e-02  3.59773093e-01
 -2.96676555e-01 -1.22358

In [79]:
print(data_testing['DAX Volume'][:50])

Date
2017-04-04    114566700.0
2017-04-05     84216400.0
2017-04-06     74256200.0
2017-04-07     71891000.0
2017-04-10     56772300.0
2017-04-11     81596000.0
2017-04-12     74185200.0
2017-04-13     73577600.0
2017-04-18     87088100.0
2017-04-19     83247900.0
2017-04-20     82816800.0
2017-04-21    106406900.0
2017-04-24    186060600.0
2017-04-25     91489400.0
2017-04-26     87667300.0
2017-04-27    101179400.0
2017-04-28    101811900.0
2017-05-02     85263200.0
2017-05-03     79318800.0
2017-05-04    102442000.0
2017-05-05    104480300.0
2017-05-08     97719300.0
2017-05-09    109857600.0
2017-05-10    104830700.0
2017-05-11    100141500.0
2017-05-12    106195300.0
2017-05-15     84466400.0
2017-05-16     77912000.0
2017-05-17    105679600.0
2017-05-18    119778200.0
2017-05-22     73068700.0
2017-05-23     88301200.0
2017-05-24     71515700.0
2017-05-25     57697300.0
2017-05-30     72307400.0
2017-05-31    136040800.0
2017-06-01     87581100.0
2017-06-02     88411500.0
2017-06